# Preprocessing after fMRIprep (using AFNI & FSL)

Code created: August 22, 2023, Hayoung Song (hyssong@uchicago.edu)

In [ ]:
import os
import numpy as np
import pandas as pd
# dependencies: AFNI & FSL (fMRI processing softwares)

## Setup

In [ ]:
dirname_fmriprep   = './fmriprepoutput'
dirname_preprocess = './preprocess'
if os.path.exists(dirname_preprocess)==0:
    os.mkdir(dirname_preprocess)

TR = 1.5

# get subject list in fMRIprep output folder
flist = [i for i in np.sort(os.listdir(dirname_fmriprep)) if i.startswith('sub-')]
tasklist = ['PreExposure_run-01','PreTest_run-01','Repeated_run-01','Repeated_run-02','Repeated_run-03',
            'Repeated_run-04','Repeated_run-05','Repeated_run-06','PostExposure_run-01','PostTest_run-01']
print(flist)

## Quality check: Head motion
This section loops over all participants' all task runs to check for head motion.
This is used as a quality check; no files are processed/generated.

In [ ]:
fd_thres = 0.5
for si, subname in enumerate(flist):
    print(subname)
    
    for ti, task in enumerate(tasklist):
        t = pd.read_csv(dirname_fmriprep+'/'+subname+'/'+'/func/'+subname+'_task-'+task+'_desc-confounds_timeseries.tsv',
                        delimiter='\t')
        
        # time course of framewise displacement (FD)
        fd = t['framewise_displacement']
        
        # proportion of frames where FD >= fd_thres
        outliers = np.sum(fd >= fd_thres) / (len(fd)-np.sum(pd.isna(fd)))
        
        print('   meanFD: '+str(np.round(np.nanmean(fd),2))+', %FD>='+str(fd_thres)+': '+str(np.round(outliers*100, 2))+'% ['+task+']')

## Creating outlier & confound matrices
This section creates "preprocess" folder for each participant/run and save outlier vector and confound regressor matrix as .1D files. 

In [ ]:
# which variables to regress out? column names of *_desc-confounds_timeseries.tsv
variables=['trans_x', 'trans_y', 'trans_z', 'rot_x', 'rot_y', 'rot_z',
           'trans_x_derivative1', 'trans_y_derivative1', 'trans_z_derivative1',
           'rot_x_derivative1', 'rot_y_derivative1', 'rot_z_derivative1',
           'trans_x_derivative1_power2', 'trans_y_derivative1_power2',
           'trans_z_derivative1_power2', 'rot_x_derivative1_power2',
           'rot_y_derivative1_power2', 'rot_z_derivative1_power2',
           'trans_x_power2', 'trans_y_power2', 'trans_z_power2', 'rot_x_power2',
           'rot_y_power2', 'rot_z_power2', 'global_signal', 'csf', 'white_matter']
# 24 motion parameters, global signal, CSF, white matter

In [ ]:
for si, subname in enumerate(flist):
    print(subname)
    if os.path.exists(dirname_preprocess+'/'+subname)==0:
        os.mkdir(dirname_preprocess+'/'+subname)
    
    for ti, task in enumerate(tasklist):
        t = pd.read_csv(dirname_fmriprep+'/'+subname+'/'+'/func/'+subname+'_task-'+task+'_desc-confounds_timeseries.tsv',
                        delimiter='\t')
        
        # ------ outliers ------ #
        # head motion outliers (FD, DVARS) & non-steady-state outliers
        # default fMRIprep outlier detection: exceeding 0.5 mm FD or 1.5 standardized DVARS
        outlier_columns = [i for i in t.columns if i.startswith('motion_outlier') or i.startswith('non_steady_state_outlier')]
        
        # time points to be included (1), to be excluded (0)
        outliers = np.zeros((len(t),))+np.nan
        outliers[np.where(np.sum(np.array(t[outlier_columns]),1)>0)[0]] = 0
        outliers[np.where(np.sum(np.array(t[outlier_columns]),1)==0)[0]] = 1
        
        # save outliers file
        if os.path.exists(dirname_preprocess+'/'+subname+'/task-'+task)==0:
            os.mkdir(dirname_preprocess+'/'+subname+'/task-'+task)
        pd.DataFrame(outliers).to_csv(dirname_preprocess+'/'+subname+'/task-'+task+'/'+subname+'_task-'+task+'_outliers.1D', 
                                      index=False, header=False)
        
        # ------ regressors ------ #
        # dupliace values in second row if the first row is NaN
        for varb in variables:
            if pd.isna(t[varb][0]):
                t[varb][0] = t[varb][1]
        regressors = t[variables]
        
        # save regressors file
        regressors.to_csv(dirname_preprocess+'/'+subname+'/task-'+task+'/'+subname+'_task-'+task+'_confounds_regressors.1D', 
                          index=False, header=False)
        regressors.to_csv(dirname_preprocess+'/'+subname+'/task-'+task+'/'+subname+'_task-'+task+'_confounds_regressors.csv')
        

## Run preprocessing
This section runs additional preprocessing on the fMRIprep outputs. Dependencies: AFNI & FSL. <br>

1. Apply brain mask to the preprocessed EPI
2. Intensity normalization
3. Regressing out nuissance variables: 24 motion parameters, WM, CSF, global signal, low-frequency signals, linear trend
4. Spatial smoothing

In [ ]:
FWHM = 5 # smoothing factor: if voxel size = 2mm, usually fwhm = 4, if voxel size = 3mm, usually fwhm = 5-6

for si, subname in enumerate(flist):
    for ti, task in enumerate(tasklist):
        print(subname+' '+task)
        
        # ------ setup ------ #
        # fmriprep directory
        prep_dir = dirname_fmriprep+'/'+subname+'/func/'
        # preprocessed output directory
        proc_dir = dirname_preprocess+'/'+subname+'/task-'+task
        
        # preprocessed, MNI-registered EPI
        prep_img = prep_dir+subname+'_task-'+task+'_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz'
        # brain mask
        prep_mask_img = prep_dir+subname+'_task-'+task+'_space-MNI152NLin2009cAsym_desc-brain_mask.nii.gz'
        
        # ------------ #
        print('1. Mask fMRIprep output')
        os.system('3dcalc -a '+prep_img+' -b '+prep_mask_img+' -expr "(a*b)" -prefix '+proc_dir+'/1_'+subname+'_task-'+task+'_fmriprep_output.nii.gz')
        
        # ------------ #
        print('2. Intensity normalization')
        os.system('fslmaths '+proc_dir+'/1_'+subname+'_task-'+task+'_fmriprep_output.nii.gz -inm 10000 '+proc_dir+'/2_'+subname+'_task-'+task+'_intensity_normalization.nii.gz')
        
        # ------------ #
        print('3. Create low-frequency signal regressors for high-pass filtering')
        # 128 s cutoff
        os.system('1dBport -input '+proc_dir+'/2_'+subname+'_task-'+task+'_intensity_normalization.nii.gz -TR '+
                 str(TR)+' -band 0 0.0078 -nozero > '+proc_dir+'/tmp_rm.hpass.1D')
        os.system('1d_tool.py -infile '+proc_dir+'/tmp_rm.hpass.1D -write '+proc_dir+'/'+subname+'_task-'+task+'_highpass_regressors.1D')
        os.system('rm '+proc_dir+'/tmp_rm.hpass.1D')
        
        # ------------ #
        print('4. Create regressors')
        # creating *_xmat.1D matrix that combines low-frequency signals, linear trend, fMRIprep output confound matrix, and an intercept
        os.system('3dDeconvolve -input '+proc_dir+'/2_'+subname+'_task-'+task+'_intensity_normalization.nii.gz '+
                  '-ortvec '+proc_dir+'/'+subname+'_task-'+task+'_highpass_regressors.1D highpass '+
                  '-ortvec '+proc_dir+'/'+subname+'_task-'+task+'_confounds_regressors.1D confounds '+
                  '-polort 1 '+
                  '-fout -tout -x1D '+proc_dir+'/'+subname+'_task-'+task+'_xmat.1D '+
                  '-fitts fitts -errts errts -x1D_stop -bucket stats')
        
        # ------------ #
        print('5. Regressing out nuissance variables')
        # inputs
        #    regressors: *_xmat.1D
        #    censor frames: *_outliers.1D (1: included frames, 0: excluded frames)
        #    (censored frames are treated as ZEROS: this should be converted to NaNs during analyses)
        # outputs
        #    3_*_nuissance_regressed.nii.gz
        os.system('3dTproject -polort 0 -input '+proc_dir+'/2_'+subname+'_task-'+task+'_intensity_normalization.nii.gz '+
                  '-ort '+proc_dir+'/'+subname+'_task-'+task+'_xmat.1D '+
                  '-censor '+proc_dir+'/'+subname+'_task-'+task+'_outliers.1D '+
                  '-cenmode ZERO '+
                  '-prefix '+proc_dir+'/3_'+subname+'_task-'+task+'_nuissance_regressed.nii.gz')
        
        # ------------ #
        print('6. Spatial smoothing with FWHM = '+str(FWHM))
        os.system('3dmerge -quiet -1blur_fwhm '+str(FWHM)+' -doall -prefix '+proc_dir+'/4_'+subname+'_task-'+task+'_smoothed.nii.gz '+
                 proc_dir+'/3_'+subname+'_task-'+task+'_nuissance_regressed.nii.gz')
        print('')
        